In [1]:
%load_ext autoreload
%autoreload 2
import sys
import sqlite3
sys.path.append("../")
from pandasdb.connections import SQLiteConnection
from pandasdb import DataBase
import pandas as pd

In [42]:
db = DataBase(SQLiteConnection(location="data/test.db"))
db.info.query

'SELECT info.id, info.buzzword, info.car, info.city, info.creditcard, info.app_version, info.country, info.lat, info.lon, info.title FROM info'

In [43]:
db.info.filter((db.info.lon < 40) & (db.info.lon > 38)).query

'SELECT info.id, info.buzzword, info.car, info.city, info.creditcard, info.app_version, info.country, info.lat, info.lon, info.title FROM info (info.lon < 40 AND info.lon > 38)'

In [7]:
db.info.select_expr(db.info.car, db.info.city, "lat as latitude").order_by("lat", False)

,car,city,latitude
0,Volkswagen,Vidyayevo,68.883330
1,Chevrolet,Kiruna,67.825618
2,Pontiac,Luleå,65.592145
3,Mercury,Luleå,65.575991
4,Bentley,Trondheim,63.410769


In [21]:
df = db.users.order_by("last_name").df()


,id,first_name,last_name,email,gender,ip_address
0,134,Oswell,Aaronson,oaaronson3p@businessweek.com,Male,66.203.161.184
1,856,Fielding,Aberkirder,faberkirdernr@sciencedaily.com,Male,196.120.238.192
2,45,Merralee,Aberkirdo,maberkirdo18@bloglovin.com,Female,202.3.255.47
3,191,Kamilah,Abramchik,kabramchik5a@admin.ch,Female,27.52.209.162
4,924,Cross,Abry,cabrypn@wunderground.com,Male,84.202.122.207
5,173,Paxton,Adamolli,padamolli4s@huffingtonpost.com,Male,197.37.161.119
6,587,Lemmy,Addinall,laddinallga@slashdot.org,Male,50.119.143.3
7,721,Angel,Adlem,aadlemk0@mediafire.com,Male,166.62.99.198
8,812,Claudell,Adnams,cadnamsmj@biblegateway.com,Male,99.231.208.110
9,758,Teddie,Adriaens,tadriaensl1@tuttocitta.it,Female,32.44.171.102


In [45]:
df = (db.info.join(db.users, "id", "id")
        .filter((db.info.car == "Buick") & (db.users.gender == "Female"))
        .select(db.info.car, db.info.app_version, db.users.gender, db.users.id)
        .order_by("app_version", ascending=False))
df.query

"SELECT info.car, info.app_version, users.gender, users.id FROM info LEFT JOIN users ON info.id = users.id (info.car = 'Buick' AND users.gender = 'Female') ORDER BY app_version DESC"

In [22]:
conn = sqlite3.connect("data/test.db")
for row in conn.cursor().execute('SELECT info.id, city, lat FROM info ORDER BY lat DESC').fetchall():
    pass

In [32]:
import pandasdb.functions as funcs
db.users.group_by("gender").agg(
    funcs.count(db.users.id).alias("amount"),
    funcs.max(db.users.id).alias("biggest_id"),
)

,gender,amount,biggest_id
0,Female,468,998
1,Male,532,1000


In [38]:
db.users.group_by("gender").agg(
    funcs.count(db.users.id).alias("amount"),
    funcs.max(db.users.id).alias("biggest_id"),
)

,gender,amount,biggest_id
0,Female,468,998
1,Male,532,1000
